In [1]:
# Load the NG data and the response function and dump output into variables
from IPython.utils import io
with io.capture_output() as output_03:
    %run '03b_bear_necessities.ipynb'
with io.capture_output() as output_05:
    %run '05_response function.ipynb'
# with io.capture_output() as output_10:
#     %run '10_read_andreas_result.ipynb'
# with io.capture_output() as output_12:
#     %run '12_read_simu.ipynb'
# with io.capture_output() as output_14:
#     %run '14_a_closer_look_at_acceptance.ipynb'

### Define functions

We define a couple of functions here. These assume a (combination of some) Gaussian in neutron incident energy. For the convolution, we need to sample in neutron_energies list, but for the fit, we need to resample to match the energy scale in measurement. This is split into two functions. Clear? For a neutron spectrum, just plot a Gauss!

In [4]:
def get_neutron_spectrum(mu, sigma):
    '''
    Recoil space.
    Get the smeared version of the response for an original neutron spectrum that is Gaussian in energy.
    Mu and sigma in MeV please.
    Requires globally defined 'neutron_energies', 'electron_energies' and 'response'
    '''
    # The parameter 'electron_energies'  comes from the presupplied response function 
    smeared_response = np.zeros(len(electron_energies))
    
    # Loop over all neutron energy bins expect for last one
    for i in range(len(neutron_energies) - 1):
        # Current neutron energy
        neutron_energy = neutron_energies[i]
        # Current bin width
        bin_width = neutron_energies[i+1] - neutron_energies[i]
        # Current amplitude in the neutron spectrum
        ampl = gaussian(neutron_energy, mu, sigma)
        # Full response for this neutron energy bin
        smeared_response += bin_width*ampl*response[i]
    return smeared_response


def get_neutron_spectrum_interpolated(energy_list, a, mu, sigma):
    '''
    Recoil space
    Interpolate the function that we get from 'get_neutron_spectrum'  to fit 'energy_list'
    '''
    smeared_response = get_neutron_spectrum(mu, sigma)
    return a * np.interp(energy_list, 1000*electron_energies, smeared_response)

def get_neutron_spectrum_interpolated_2(energy_list, a1, mu1, sigma1, a2, mu2, sigma2):
    '''
    Two copies of get_neutron_spectrum_interpolated for fitting of two energy populations
    '''
    smeared_response1 = get_neutron_spectrum(mu1, sigma1)
    smeared_response2 = get_neutron_spectrum(mu2, sigma2)
    return ( a1 * np.interp(energy_list, 1000*electron_energies, smeared_response1) +
            a2 * np.interp(energy_list, 1000*electron_energies, smeared_response2))

def get_neutron_spectrum_interpolated_3(energy_list, 
                                        a1, mu1, sigma1,
                                        a2, mu2, sigma2, 
                                        a3, mu3, sigma3):
    '''
    Three copies of get_neutron_spectrum_interpolated for fitting of three energy populations
    '''
    smeared_response1 = get_neutron_spectrum(mu1, sigma1)
    smeared_response2 = get_neutron_spectrum(mu2, sigma2)
    smeared_response3 = get_neutron_spectrum(mu3, sigma3)
    return ( a1 * np.interp(energy_list, 1000*electron_energies, smeared_response1) +
            a2 * np.interp(energy_list, 1000*electron_energies, smeared_response2) +
           a3 * np.interp(energy_list, 1000*electron_energies, smeared_response3))

def get_chi_sq(y_data, y_err, y_fit):
    '''
    This function is only used in 'manual minimization' so it is depricated
    '''
    print('DepricationWarning')
    return sum((y_fit - y_data)**2/(y_err**2))

def draw_box(x, y, **kwargs):
    """Draw rectangle, given x-y boundary tuples"""
    import matplotlib as mpl
    # Arcane syntax of the week: matplotlib's Rectangle...
    plt.gca().add_patch(mpl.patches.Rectangle(
        (x[0], y[0]), x[1] - x[0], y[1] - y[0], **kwargs))
    


It gets a bit more complicated: we want to do this little bit for ANY neutron spectrum, specified by an energy spectrum (that follows distribution according to neutron_energies, again!). 

In [5]:

def get_neutron_spectrum_custom(amp_list):
    '''
    Recoil space
    Use a custom list of amplitudes
    Mu and sigma in MeV please.
    Requires globally defined 'neutron_energies', 'electron_energies' and 'response'
    '''
    
    assert(len(amp_list) == len(neutron_energies))
    
    # The parameter 'electron_energies'  comes from the presupplied response function 
    smeared_response = np.zeros(len(electron_energies))
    
    # Loop over all neutron energy bins expect for last one
    for i in range(len(neutron_energies) - 1):
        # Current neutron energy
        neutron_energy = neutron_energies[i]
        # Current bin width
        bin_width = neutron_energies[i+1] - neutron_energies[i]
        # Current amplitude in the neutron spectrum
        ampl = amp_list[i]
        # Full response for this neutron energy bin
        smeared_response += bin_width*ampl*response[i]
    return smeared_response

def get_neutron_spectrum_custom_interpolated(energy_list, amp_list):
    '''
    Recoil space
    Amp_list follows neutron_energies!
    Output is y to whatever x you specify with energy_list (how about e_scale?)
    '''
    smeared_response = get_neutron_spectrum_custom(amp_list)
    return np.interp(energy_list, 1000*electron_energies, smeared_response)